<a href="https://colab.research.google.com/github/INT-Group-9/ColabNotebook/blob/Harry/cifa10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [2]:
trainSet = datasets.CIFAR10("./data", train=True, download = True,
                         transform = transforms.Compose([
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomCrop(32, padding=4),
                            transforms.ToTensor()]))

testSet = datasets.CIFAR10("./data", train=False, download=True,
                         transform = transforms.Compose([transforms.ToTensor()]))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=True)

In [4]:
class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.batch1 = nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.batch2 = nn.BatchNorm2d(128)

    self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.batch3 = nn.BatchNorm2d(256)


    self.fc1 = nn.Linear(in_features = 256 * 4 * 4, out_features=1024)
    self.fc2 = nn.Linear(in_features = 1024, out_features=512)
    self.fc3 = nn.Linear(in_features = 512, out_features=10)
    self.dropout1 = nn.Dropout(p=0.1)
    self.dropout2 = nn.Dropout(p=0.05)

    self.pool = nn.MaxPool2d(2, 2)

  def forward(self, input):
    output = self.conv1(input)
    output = self.batch1(output)
    output = F.relu(output)
    output = F.relu(self.conv2(output))
    output = self.pool(output)

    output = self.conv3(output)
    output = self.batch2(output)
    output = F.relu(output)
    output = F.relu(self.conv4(output))
    output = self.pool(output)
    output = self.dropout2(output)

    output = self.conv5(output)
    output = self.batch3(output)
    output = F.relu(output)
    output = F.relu(self.conv6(output))
    output = self.pool(output)

    output = output.view(output.size(0), -1)

    output = self.dropout1(output)
    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.dropout1(output)
    output = self.fc3(output)

    return output
    
net = Net().cuda()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [6]:
import tqdm

epochs = 80

for epoch in range(epochs):  # loop over the dataset multiple times

    print("epoch", epoch + 1)

    for i, data in enumerate(tqdm.notebook.tqdm(trainLoader), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

epoch 1



epoch 2



epoch 3



epoch 4



epoch 5


...


epoch 77



epoch 78



epoch 79



epoch 80



Finished Training


In [24]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [49]:
from sklearn import metrics

total = 0
correct = 0
everyPred = torch.tensor([]).cuda()
everyLabel = torch.tensor([]).cuda()

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)

        # calculations for the accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # calculations for confusion matrix
        everyLabel = torch.cat((everyLabel, labels), dim=0)
        everyPred = torch.cat((everyPred, predicted), dim=0)


print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total), '\n')

print(metrics.confusion_matrix(everyLabel.cpu(), everyPred.cpu()), '\n')

print(metrics.classification_report(everyLabel.cpu(), everyPred.cpu(), labels=[0,1,2,3,4,5,6,7,8,9], target_names=classes))

Accuracy of the network on the 10000 test images: 89 % 

[[932   1  20   8   4   1   2   3  24   5]
 [ 10 944   1   1   1   0   0   1  11  31]
 [ 24   1 870  23  26  15  25  12   4   0]
 [ 13   1  25 780  28  87  28  17  14   7]
 [  8   1  24  20 888  12  20  25   1   1]
 [  5   1  26  86  18 818   7  31   5   3]
 [  7   1  13  22  10   7 931   3   4   2]
 [ 10   0   9  14  11   7   0 943   0   6]
 [ 39   4   4   3   0   2   0   1 942   5]
 [ 15  34   4   3   0   0   2   1  23 918]] 

              precision    recall  f1-score   support

       plane       0.88      0.93      0.90      1000
         car       0.96      0.94      0.95      1000
        bird       0.87      0.87      0.87      1000
         cat       0.81      0.78      0.80      1000
        deer       0.90      0.89      0.89      1000
         dog       0.86      0.82      0.84      1000
        frog       0.92      0.93      0.92      1000
       horse       0.91      0.94      0.93      1000
        ship       0.92